###### Lab06 Multivariate Linear Regression

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [2]:
# 데이터
x_train = torch.FloatTensor([[73,80,75],
                            [93,88,93],
                            [89,91,90],
                            [96,98,100],
                            [73,66,70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W,b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371071
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch   10/20 hypo

## nn.Module 적용

In [3]:
# 데이터
x_train = torch.FloatTensor([[73,80,75],
                            [93,88,93],
                            [89,91,90],
                            [96,98,100],
                            [73,66,70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
# W = torch.zeros((3,1), requires_grad=True)
# b = torch.zeros(1, requires_grad=True)
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self,x):
        return self.linear(x)

model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    # hypothesis = x_train.matmul(W) + b
    hypothesis = model(x_train)
    
    # cost 계산
    # cost = torch.mean((hypothesis - y_train) ** 2)
    cost = F.mse_loss(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([-64.6765, -76.7383, -76.2648, -82.0162, -59.0178]) Cost: 59765.691406
Epoch    1/20 hypothesis: tensor([30.7955, 38.0127, 36.8010, 41.1096, 28.5084]) Cost: 18734.623047
Epoch    2/20 hypothesis: tensor([ 84.2467, 102.2577, 100.1023, 110.0432,  77.5114]) Cost: 5873.562012
Epoch    3/20 hypothesis: tensor([114.1718, 138.2262, 135.5423, 148.6365, 104.9465]) Cost: 1842.302490
Epoch    4/20 hypothesis: tensor([130.9257, 158.3638, 155.3839, 170.2434, 120.3065]) Cost: 578.715698
Epoch    5/20 hypothesis: tensor([140.3054, 169.6382, 166.4923, 182.3403, 128.9062]) Cost: 182.648560
Epoch    6/20 hypothesis: tensor([145.5566, 175.9504, 172.7115, 189.1128, 133.7210]) Cost: 58.501778
Epoch    7/20 hypothesis: tensor([148.4964, 179.4845, 176.1934, 192.9045, 136.4169]) Cost: 19.588036
Epoch    8/20 hypothesis: tensor([150.1421, 181.4632, 178.1427, 195.0272, 137.9263]) Cost: 7.390363
Epoch    9/20 hypothesis: tensor([151.0633, 182.5712, 179.2340, 196.2156, 138.7715]) 

## Mini-batch Gradient Descent ft. Dataset, DataLoader

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73,80,75],
                        [93,88,93],
                        [89,91,90],
                        [96,98,100],
                        [73,66,70]]
        self.y_data = [[152],[185],[180],[196],[142]]
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x,y
    
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        # H(x) 계산
        # hypothesis = x_train.matmul(W) + b
        hypothesis = model(x_train)
    
        # cost 계산
        # cost = torch.mean((hypothesis - y_train) ** 2)
        cost = F.mse_loss(hypothesis, y_train)
    
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        print('Epoch {:4d}/{} Batch {}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader), hypothesis.squeeze().detach(), cost.item()
        ))

Epoch    0/20 Batch 1/3 hypothesis: tensor([183.6709, 180.2876]) Cost: 0.924643
Epoch    0/20 Batch 2/3 hypothesis: tensor([197.6438, 152.1473]) Cost: 1.361816
Epoch    0/20 Batch 3/3 hypothesis: 139.46475219726562 Cost: 6.427482
Epoch    1/20 Batch 1/3 hypothesis: tensor([181.0336, 152.5586]) Cost: 0.690169
Epoch    1/20 Batch 2/3 hypothesis: tensor([197.7770, 139.9214]) Cost: 3.739195
Epoch    1/20 Batch 3/3 hypothesis: 183.97857666015625 Cost: 1.043306
Epoch    2/20 Batch 1/3 hypothesis: tensor([140.2510, 198.2391]) Cost: 4.036129
Epoch    2/20 Batch 2/3 hypothesis: tensor([180.8284, 184.2231]) Cost: 0.644898
Epoch    2/20 Batch 3/3 hypothesis: 152.37554931640625 Cost: 0.141037
Epoch    3/20 Batch 1/3 hypothesis: tensor([152.2452, 180.6644]) Cost: 0.250793
Epoch    3/20 Batch 2/3 hypothesis: tensor([183.8423, 197.5428]) Cost: 1.860181
Epoch    3/20 Batch 3/3 hypothesis: 139.66265869140625 Cost: 5.463164
Epoch    4/20 Batch 1/3 hypothesis: tensor([152.7037, 184.6112]) Cost: 0.323197
